# Task 3: Simulation of a GBS experiment

A template code for calculating a vibronic spectrum of $V_3$ using a sampling method, simulating a Gaussian Boson Sampling experiment.

Input Parameters.

Note: "Strawberry Fields" is a code created by Xanadu and is the driver behind GBS. This can be installed using **pip**, but see install instructions in the <a href="https://github.com/CDL-Quantum/CohortProject_2020/blob/master/Project_3_Franck_Condon_Factors/Project3_LandingPage.pdf"> Project 3 Landing Page</a>. Plotly is a nice plotting code, but is not necessary if you wish to use one of your choice.

In [1]:
from sys import argv
from strawberryfields.apps import vibronic, data, sample, plot
import numpy as np
from plotly import offline
offline.init_notebook_mode(connected = True)

Variables.

Note: The **inputfile** for $V_3$ is what you will have to create to run this script.

In [2]:
# inputfile = open(argv[1], "r")
inputfile = open("Task2Code/output/V3.duschinsky.out", "r")

N=int(inputfile.readline())     #Number of Atoms
nmodes=3*N-6                    #Number of Modes

w=np.zeros(nmodes,float)        #vib. frequencies of ground electronic state
wp=np.zeros(nmodes,float)       #vib. frequencies of excited electronic state
Ud=np.zeros((nmodes,nmodes),float)   #Duschinsky Matrix
delta=np.zeros(nmodes,float)    #Displacement Vector

T = 500  # temperature

Read in Parameters.

The code below assumes, that the input file has the following structure
```
1 line: number of atoms
1 line: space-separated list of normal frequencies of the ground state
1 line: space-separated list of normal frequencies of the excited state
nmodes lines: each line space-separated numbers representing a row in Duschinsky matrix
1 line: space-separated list of displacements
```

In [3]:
w = np.array([float(x) for x in inputfile.readline().split()])
wp = np.array([float(x) for x in inputfile.readline().split()])
for i in range(nmodes):
    Ud[i, :] = np.array([float(x) for x in inputfile.readline().split()])
delta = np.array([float(x) for x in inputfile.readline().split()])

Use "Strawberry Fields" to calculate GBS Parameters based on the frequencies, Duschinsky matrix, and temperature that was provided as input.

In [4]:
t, U1, r, U2, alpha = vibronic.gbs_params(w, wp, Ud, delta, T)

Generate Samples and Calculate Energies of Samples.

Note: The number of samples may be changed if the spectrum is not converged.

In [5]:
nr_samples = 100
s = sample.vibronic(t, U1, r, U2, alpha, nr_samples)
e = vibronic.energies(s, w, wp)

Plot the Spectrum

Note: You may choose your favourite plotting program if you wish.

In [6]:
spectrum = plot.spectrum(e, xmin=-300, xmax=2000)
offline.iplot(spectrum)